In [47]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import numpy as np
import scipy.stats
#import seaborn as sns

from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
import time
import pytz

import ipywidgets

%matplotlib inline

In [48]:
def Get_data(tickers,inicio,fim,janela):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")
    
    #Verifica caso seja timeframe diario ou intraday
    if janela == '1d':
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_D1
    #aqui trata as datas intraday
    else:
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], inicio[3], inicio[4], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], fim[3], fim[4], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_M15
        
    # obtendo o par de acoes com base nas datas enviadas nas strings, caso len seja 1, getdata retorna o dataframe de uma unica acao
    if (len (tickers) == 1):        
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)    
        stock_df = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
                
    else:    
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)
        stock2 = mt5.copy_rates_range(tickers[1], timeframe, dt_inicio, dt_fim)
        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})
        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    
    #Se o timeframe for intraday, retorna um datetime (com minutos e horas), caso contrario, retorna um date
    if (janela != '1d'):
        stock_df['Date'] = stock_df['time'].apply(lambda x: str (datetime.fromtimestamp(x, tz=timezone))[:-9]) #define timezone
    else:
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)
        
    return stock_df.set_index('Date')

In [49]:
# Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
def regressao_multivariada(x, y, periodo):  
    
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore, res


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    return res.params[1]

# ADF da Statsmodel e teste de significancia com os critical values do Ferro
def adftest(df,reg):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    adf_stat = adf[0]
    adf_pvalue = adf[1]
    
    ## %ADF clássico, de acordo com o output da função  - NAO UTILIZADO
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    ## %ADF do Ferro
    # Utiliza duas tabelas - uma para quando o tempo é significante, outra quando não é (indep. do noobs)
    ttest = reg.params[1]/reg.bse[1]
    critical_value_tempo = scipy.stats.t.ppf(0.01,df=(len(df)-3)) # Retorna o inverso bicaudal da distribuição t de Student

    if (abs(ttest) < critical_value_tempo): aceitar_t0 = '99%'
    else:  aceitar_t0 = '0%'
    
    if (aceitar_t0 == '0%'):
        if   adf[0] < -4.32:         adf_sign='99%'
        elif adf[0] < -3.67:         adf_sign='95%'
        elif adf[0] < -3.28:         adf_sign='90%'
        else:                        adf_sign='-' 
    else:    
        if   adf[0] < -3.58:         adf_sign='99%'
        elif adf[0] < -3.22:         adf_sign='95%'
        elif adf[0] < -2.60:         adf_sign='90%'
        else:                        adf_sign='-'                 

    
    
        
    return adf_stat, adf_sign, adf_pvalue  


def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = float(x.tail(1))
    ultimo_y = float(y.tail(1))
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv


def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return (int(round(half_life)))


In [50]:
def ajusta_B3Date (data_start, qtde_dias):
    
    df = pd.read_excel('Calendar.xlsx')
    df['Date'] = df['Date'].apply(str)
    index = df [df['Date'] == data_start].index.values.astype(int)[0]
    data_fim = df.iloc[index+qtde_dias][1]
    
    return (data_fim + '-17-00')

    
def simula_trade (ticker, beta_inicio, dp_entrada, dp_target, start_time , hlife):
    
    #passando a data de inicio do trade para identificarmos qual a data limite para o trade com base no calendario da B3
    end_time = ajusta_B3Date(start_time[:-6],hlife)
    #print (start_time)
    #print (end_time)
    stock = Get_data(ticker, start_time, end_time,'M5')
    
    return stock
    
def merge_intraday (ticker, dt_inicio, dt_intraday):

    dt_temp = dt_intraday.split('-')
    dt_temp = [int(element) for element in dt_temp]
    dt_temp[2] = dt_temp[2] - 1
    dt_fim = str(dt_temp[0])+'-0'+str(dt_temp[1])+'-'+str(dt_temp[2])
    stock  = Get_data(ticker, dt_inicio,dt_fim,'1d')
    stock_intraday = Get_data(ticker, dt_intraday,dt_intraday,'M5')
    stock_intraday['Date'] =  stock_intraday['time'].apply(date.fromtimestamp)
    stock_intraday.set_index('Date', inplace = True)
    stock_concat = pd.concat([stock,stock_intraday])
    stock_concat.drop('time',axis=1,inplace = True)

    return stock_concat

In [99]:
def Get_Cubo (tickers, dt_cubo, intra_bt):
    
    #Pegando a data B3 de 250 regoes atras
    if intra_bt:
        dt_inicio = ajusta_B3Date(dt_cubo[:-6],-250)
    else:
        dt_inicio = ajusta_B3Date(dt_cubo,-250)
    
    #print (dt_inicio)
    
    if intra_bt:
        #Para retornar o par com dados diarios, com ultimo dia sendo o intraday passado em dt_final
        par = merge_intraday (tickers, dt_inicio, dt_cubo)
    else:   
        # Busca cotacoes no Yahoo Finance
        par = Get_data(tickers, dt_inicio, dt_cubo,'1d')

    par['ratio']  = par[tickers[0]]/par[tickers[1]]
    par['ratio1'] = par[tickers[1]]/par[tickers[0]]

    # Cria o cubo de periodos
    cubo = pd.DataFrame(columns=['periodo','ratio','adf_stat','adf_sign','coef_Ang',
                                 'desvio','pct_fin','meia_vida', 'corr'])

    # Popula o cubo de periodos
    #for i in range(100,260,10):
    for i in [100,120,140,160,180,200,220,240,250]:

        # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
        par_subset = par.tail(i)
        x = par_subset[tickers[1]]
        y = par_subset[tickers[0]]

        # Adiciona o PERIODO analisado no dataframe
        cubo = cubo.append({'periodo': int(i)}, ignore_index=True)
        # Gera a REGRESSAO e retorna o Coef Ang, os residuos e o z-score. 
        # Retorna tbm o vetor de retorno da regressao para ser usado no ADF
        coef_ang, residuo, zscore, reg  = regressao_multivariada(x, y, i)

        # Teste de Estacionariedade dos Resíduos (ADF)
        adf_stat,adf_sign,adf_pvalue = adftest(residuo, reg)
         
        # Calculo do Fisher
        fisher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.diff().fillna(method="bfill"), y.diff().fillna(method="bfill"))

        # Calculo do % Financeiro (C/V)
        pct_fin = pct_financeiro(x,y,coef_ang, float(zscore.tail(1)))

        # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
        meia_vida = calculo_meia_vida1(residuo)

        # Correlação 
        corr = np.corrcoef(x.pct_change().cumsum().dropna(), y.pct_change().cumsum().dropna())[1,0]

        # Se fisher minimo menor que 10%, ou se correlacao < 0, faz adf_sign=0%
        if ((fisher_lo <= 0.1) or (corr<=0)):
            adf_sign = "0%"
            
        # Preenchimento do cubo
        cubo.loc[ (cubo['periodo']==i), 'ratio'      ] = float(y.tail(1))/float(x.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'coef_Ang'   ] = coef_ang 
        cubo.loc[ (cubo['periodo']==i), 'desvio'     ] = float(zscore.tail(1))
        cubo.loc[ (cubo['periodo']==i), 'adf_stat'   ] = adf_stat
        cubo.loc[ (cubo['periodo']==i), 'adf_sign'   ] = adf_sign 
        cubo.loc[ (cubo['periodo']==i), 'pct_fin'    ] = pct_fin
        cubo.loc[ (cubo['periodo']==i), 'meia_vida'  ] = meia_vida
        cubo.loc[ (cubo['periodo']==i), 'corr'       ] = corr


    #par.to_excel("Par.xlsx")    
    #display(cubo)


    
    return cubo

In [104]:
tickers = ['ITUB4','BBSE3']
a = Get_Cubo(tickers,'2020-08-10',False)

In [114]:


# Cubo de fechamentos
datas = Get_data(tickers,'2018-01-01','2020-08-10','1d').reset_index()['Date']

tmp = pd.DataFrame()
# cubo_fechamentos = pd.DataFrame(columns=['data','periodo','ratio','adf_stat','adf_sign','coef_Ang',
#                                          'desvio','pct_fin','meia_vida', 'corr'])

# for dat in datas:
#     tmp = Get_Cubo(tickers,str(dat),False)
#     tmp['data']=str(dat)
#     cubo_fechamentos = cubo_fechamentos.append(tmp)

# print(cubo_fechamentos)

# Cubo intraday
datahora = Get_data(tickers,'2018-01-01-10-00','2020-08-10-17-00','15m').reset_index()['Date']

cubo_intraday = pd.DataFrame(columns=['data','periodo','ratio','adf_stat','adf_sign','coef_Ang',
                                         'desvio','pct_fin','meia_vida', 'corr'])

for dthora in datahora:
    
    # Convertendo 2018-01-02 10:30 para 2018-01-02-10-30
    dthora1 = dthora.split(' ')
    dthora2 = dthora1[0]+"-"+dthora1[1]
    dthora1 = dthora2.split(':')
    dthora2 = dthora1[0]+"-"+dthora1[1]
    print(dthora2)
    tmp = Get_Cubo(tickers,str(dthora2),True)
    tmp['data']=str(dthora2)
    cubo_intraday = cubo_intraday.append(tmp)

print(cubo_intraday)




2018-01-02-10-00
2018-01-02-10-15
2018-01-02-10-30
2018-01-02-10-45
2018-01-02-11-00
2018-01-02-11-15
2018-01-02-11-30
2018-01-02-11-45
2018-01-02-12-00
2018-01-02-12-15
2018-01-02-12-30
2018-01-02-12-45
2018-01-02-13-00
2018-01-02-13-15
2018-01-02-13-30
2018-01-02-13-45
2018-01-02-14-00
2018-01-02-14-15
2018-01-02-14-30
2018-01-02-14-45
2018-01-02-15-00
2018-01-02-15-15
2018-01-02-15-30
2018-01-02-15-45
2018-01-02-16-00
2018-01-02-16-15
2018-01-02-16-30
2018-01-02-16-45
2018-01-02-17-00
2018-01-02-17-15
2018-01-02-17-30
2018-01-02-17-45
2018-01-03-10-00
2018-01-03-10-15
2018-01-03-10-30
2018-01-03-10-45
2018-01-03-11-00
2018-01-03-11-15
2018-01-03-11-30
2018-01-03-11-45
2018-01-03-12-00
2018-01-03-12-15
2018-01-03-12-30
2018-01-03-12-45
2018-01-03-13-00
2018-01-03-13-15
2018-01-03-13-30
2018-01-03-13-45
2018-01-03-14-00
2018-01-03-14-15
2018-01-03-14-30
2018-01-03-14-45
2018-01-03-15-00
2018-01-03-15-15
2018-01-03-15-30
2018-01-03-15-45
2018-01-03-16-00
2018-01-03-16-15
2018-01-03-16-

ValueError: day is out of range for month